In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
import keras_tuner as kt

In [2]:
industries = {
    'agro': ['CPF.BK', 'CBG.BK', 'OSP.BK', 'ITC.BK', 'BTG.BK'],
    'consump': ['AURA.BK', 'STGT.BK', 'BLC.BK', 'SNPS.BK', 'SABINA.BK'],
    'fincial': ['KBANK.BK', 'KTB.BK', 'SCB.BK', 'BBL.BK', 'TISCO.BK'],
    'indus': ['PTTGC.BK', 'SCGP.BK', 'IVL.BK', 'MCS.BK', 'TSTH.BK'],
    'propcon': ['SCC.BK', 'CPN.BK', 'WHA.BK', 'TOA.BK', 'AP.BK'],
    'resource': ['PTTEP.BK', 'BCP.BK', 'PTT.BK', 'TOP.BK', 'GPSC.BK'],
    'service': ['CPALL.BK', 'AOT.BK', 'BDMS.BK', 'HMPRO.BK', 'MINT.BK'],
    'tech': ['ADVANC.BK', 'TRUE.BK', 'DELTA.BK', 'CCET.BK', 'KCE.BK']
}

def create_sequences(data, sequence_length=60):
    X, y = [], []
    for i in range(sequence_length, len(data)):
        X.append(data[i-sequence_length:i, 0])
        y.append(data[i, 0])
    return np.array(X).reshape(-1, sequence_length, 1), np.array(y)

def build_model(input_shape, lstm_units=50, dense_units=25, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(units=lstm_units, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=lstm_units, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=lstm_units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=dense_units))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def run_tuner(x_train, y_train, input_shape, project_name):
    def model_builder(hp):
        return build_model(
            input_shape=input_shape,
            lstm_units=hp.Choice('lstm_units', [32, 64, 100, 128]),
            dropout_rate=hp.Choice('dropout_rate', [0.1, 0.2, 0.3]),
            dense_units=hp.Choice('dense_units', [16, 25, 50])
        )

    tuner = kt.RandomSearch(
        model_builder,
        objective='val_loss',
        max_trials=3,
        executions_per_trial=1,
        directory='tuning_logs',
        project_name=project_name
    )

    tuner.search(x_train, y_train, validation_split=0.2, epochs=30,
                 batch_size=32, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

    best_model = tuner.get_best_models(1)[0]
    best_hp = tuner.get_best_hyperparameters(1)[0]
    return best_model, best_hp

In [3]:
# Generate tuner per industry
industry_configs = {}

for industry, stocks in industries.items():
    base_stock = stocks[0]
    print(f"\n🎯 Tuning industry config for {industry} using {base_stock}...")

    data_path = os.path.join("data", industry, f"{base_stock}.csv")
    df = pd.read_csv(data_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df = df[['Close']]

    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)
    train_data, _ = train_test_split(scaled_data, train_size=0.8, shuffle=False)
    x_train, y_train = create_sequences(train_data)

    best_model, best_hp = run_tuner(x_train, y_train, input_shape=(x_train.shape[1], 1), project_name=f"{industry}_baseline")
    industry_configs[industry] = {
        'lstm_units': best_hp.get('lstm_units'),
        'dropout_rate': best_hp.get('dropout_rate'),
        'dense_units': best_hp.get('dense_units')
    }
    print(f"✅ Best config for {industry}: {industry_configs[industry]}")

Trial 3 Complete [00h 00m 22s]
val_loss: 0.00047763032489456236

Best val_loss So Far: 0.00037706168950535357
Total elapsed time: 00h 01m 23s
✅ Best config for tech: {'lstm_units': 32, 'dropout_rate': 0.1, 'dense_units': 25}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 28 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [4]:
industry_configs

{'agro': {'lstm_units': 128, 'dropout_rate': 0.2, 'dense_units': 16},
 'consump': {'lstm_units': 128, 'dropout_rate': 0.1, 'dense_units': 25},
 'fincial': {'lstm_units': 100, 'dropout_rate': 0.2, 'dense_units': 25},
 'indus': {'lstm_units': 100, 'dropout_rate': 0.3, 'dense_units': 50},
 'propcon': {'lstm_units': 128, 'dropout_rate': 0.2, 'dense_units': 50},
 'resource': {'lstm_units': 100, 'dropout_rate': 0.1, 'dense_units': 16},
 'service': {'lstm_units': 100, 'dropout_rate': 0.3, 'dense_units': 16},
 'tech': {'lstm_units': 32, 'dropout_rate': 0.1, 'dense_units': 25}}

In [6]:
# train all the stocks using tuner for each industry
for industry, stocks in industries.items():
    for stock in stocks:
        print(f"\n📦 Training model for {stock} using {industry} config")

        data_file = os.path.join("data", industry, f"{stock}.csv")
        if not os.path.exists(data_file):
            print(f"⚠️ File not found: {data_file}")
            continue

        df = pd.read_csv(data_file)
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        df = df[['Close']]

        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(df)
        train_data, test_data = train_test_split(scaled_data, train_size=0.8, shuffle=False)

        if len(test_data) < 60:
            print(f"⚠️ Not enough test data for {stock}. Skipping...")
            continue

        x_train, y_train = create_sequences(train_data)
        x_test, y_test = create_sequences(test_data)

        if len(x_train) == 0 or len(x_test) == 0:
            print(f"⚠️ No sequences generated for {stock}. Skipping...")
            continue

        config = industry_configs[industry]
        model = build_model(
            input_shape=(x_train.shape[1], 1),
            lstm_units=config['lstm_units'],
            dropout_rate=config['dropout_rate'],
            dense_units=config['dense_units']
        )

        model_dir = os.path.join("models", industry)
        os.makedirs(model_dir, exist_ok=True)
        model_file = os.path.join(model_dir, f"{stock}_model.keras")
        scaler_file = os.path.join(model_dir, f"{stock}_scaler.pkl")

        early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
        model_checkpoint = ModelCheckpoint(model_file, monitor='loss', save_best_only=True)

        model.fit(
            x_train, y_train,
            batch_size=32,
            epochs=50,
            validation_split=0.2,
            callbacks=[early_stop, model_checkpoint]
        )

        model.save(model_file)
        with open(scaler_file, 'wb') as f:
            pickle.dump(scaler, f)
        print(f"✅ Model and scaler saved for {stock}")

print("\n✅ All models tuned, trained, and evaluated!")


📦 Training model for CPF.BK using agro config
Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 120ms/step - loss: 0.0800 - val_loss: 0.0175
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 112ms/step - loss: 0.0078 - val_loss: 0.0046
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 110ms/step - loss: 0.0060 - val_loss: 0.0033
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 126ms/step - loss: 0.0047 - val_loss: 0.0034
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - loss: 0.0049 - val_loss: 0.0041
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 139ms/step - loss: 0.0044 - val_loss: 0.0024
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 115ms/step - loss: 0.0042 - val_loss: 0.0040
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 123ms/step - loss: 0.0042 - val_loss: 0.0033
Epoch 9/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 154ms/step - loss: 0.0035 - val_loss: 0.0033
Epoch 10/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - loss: 0.0035 - val_loss: 0.0024
Epoch 11/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - loss: 0.0036 - val_loss: 0.0023
Epoch 12/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 136ms/step - loss: 0.

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 156ms/step - loss: 0.1068 - val_loss: 0.0132
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 135ms/step - loss: 0.0082 - val_loss: 0.0058
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 134ms/step - loss: 0.0063 - val_loss: 0.0021
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 141ms/step - loss: 0.0055 - val_loss: 0.0029
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 138ms/step - loss: 0.0045 - val_loss: 0.0027
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 141ms/step - loss: 0.0044 - val_loss: 0.0014
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 136ms/step - loss: 0.0056 - val_loss: 0.0028
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 140ms/step - loss: 0.0047 - val_loss: 0.0027
Epoch 9/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 152ms/step - loss: 0.0037 - val_loss: 0.0021
Epoch 10/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 139ms/step - loss: 0.0034 - val_loss: 0.0020
Epoch 11/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 144ms/step - loss: 0.0033 - val_loss: 0.0018
Epoch 12/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 141ms/step - loss: 0.

...

24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - loss: 0.0230 - val_loss: 0.0024
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0036 - val_loss: 0.0020
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0026 - val_loss: 0.0019
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0022 - val_loss: 0.0019
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0021 - val_loss: 0.0018
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.0019 - val_loss: 0.0016
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0019 - val_loss: 0.0017
Epoch 9/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0015 - val_loss: 0.0014
Epoch 10/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0014 - val_loss: 0.0019
Epoch 11/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0015 - val_loss: 0.0019
Epoch 12/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0017 - val_l

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 0.0035 - val_loss: 9.6040e-04
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 6.2164e-04 - val_loss: 5.2331e-04
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 2.5995e-04 - val_loss: 5.2829e-04
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 2.5373e-04 - val_loss: 4.7200e-04
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 2.3589e-04 - val_loss: 4.3729e-04
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 2.4249e-04 - val_loss: 4.2303e-04
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 2.2012e-04 - val_loss: 4.0271e-04
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 1.8897e-04 - val_loss: 4.8486e-04
Epoch 9/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 2.1550e-04 - val_loss: 3.7418e-04
Epoch 10/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 1.8861e-04 - val_loss: 3.4437e-04
Epoch 11/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 2.1231e-04 - val_loss: 3

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - loss: 0.1054 - val_loss: 0.0028
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0103 - val_loss: 0.0041
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0055 - val_loss: 0.0022
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0050 - val_loss: 0.0020
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0043 - val_loss: 0.0017
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0041 - val_loss: 0.0017
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0037 - val_loss: 0.0015
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0039 - val_loss: 0.0015
Epoch 9/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0040 - val_loss: 0.0017
Epoch 10/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0034 - val_loss: 0.0015
Epoch 11/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0038 - val_loss: 0.0013
Epoch 12/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0034 - val_l

In [ ]:
# --------------------------------------------------------------
# Retrain Individual

In [371]:
# Load the latest preprocessed data from CSV file
# file_path = os.path.join("data", "Fincial", "SCB.BK.csv")
import yfinance as yf
import datetime
import os

# file_path = os.path.join("data", "Indus", "PTTGC.BK.csv")

# Load and preprocess the data
# df = pd.read_csv(file_path)
df = yf.download("KCE.BK", start='2017-01-01', end='2025-04-01', auto_adjust=True) 

# df['Date'] = pd.to_datetime(df['Date'])
# df.set_index('Date', inplace=True)
df = df[['Close', 'Volume']]
# df['Return'] = df['Close'].pct_change()
# df['Volatility'] = df['Close'].rolling(window=14).std()
# df = df[['Close', 'Volume', 'Return', 'Volatility']].dropna()
# df = df[df.index >= '2021-01-01']
df

[*********************100%***********************]  1 of 1 completed


Price,Close,Volume
Ticker,KCE.BK,KCE.BK
Date,,
2017-01-04,46.941406,4009000
2017-01-05,46.184284,3180200
2017-01-06,45.805725,2405600
2017-01-09,45.805725,3384000
2017-01-10,47.130680,6695200
...,...,...
2025-03-25,18.000000,3992100
2025-03-26,17.799999,3989800


In [372]:
# Scale to range 0, 1
scaler = MinMaxScaler(feature_range=(0, 1))  # Normalize between 0 and 1
scaled_data = scaler.fit_transform(df[['Close', 'Volume']])
processed_data = pd.DataFrame(scaled_data, columns=['Close', 'Volume'])
processed_data

,Close,Volume
0,0.510972,0.019992
1,0.500417,0.015859
2,0.495140,0.011996
3,0.495140,0.016875
4,0.513611,0.033388
...,...,...
1999,0.107511,0.019908
2000,0.104723,0.019897
2001,0.096358,0.039999
2002,0.090782,0.012573


In [373]:
# Split into the training and testing sets
train_data, test_data = train_test_split(scaled_data, train_size=0.8, shuffle=False)

def create_sequences(data, sequence_length=90):
    X = []
    y = []
    for i in range(sequence_length, len(data)):
        X.append(data[i-sequence_length:i])         # All features for past 60 days
        y.append(data[i, 0])                         # Only Close price is the target (1st column)

    return np.array(X), np.array(y)

x_train, y_train = create_sequences(train_data)
x_test, y_test = create_sequences(test_data)

# Print dataset shapes
print(f"x_train shape: {x_train.shape}")  # Expecting (samples, 60, 2)
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}" if len(x_test) > 0 else "⚠️ No test set available.")
print(f"y_test shape: {y_test.shape}" if len(y_test) > 0 else "")

x_train shape: (1513, 90, 2)
y_train shape: (1513,)
x_test shape: (311, 90, 2)
y_test shape: (311,)


In [374]:
def model_builder(hp):
    model = Sequential()
    hp_units = hp.Choice('lstm_units', [32, 50, 64, 100])
    hp_dropout = hp.Choice('dropout_rate', [0.1, 0.2, 0.3])
    hp_dense = hp.Choice('dense_units', [16, 25, 50])

    model.add(LSTM(units=hp_units, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(Dropout(hp_dropout))

    model.add(LSTM(units=hp_units, return_sequences=True))
    model.add(Dropout(hp_dropout))

    model.add(LSTM(units=hp_units))
    model.add(Dropout(hp_dropout))

    model.add(Dense(units=hp_dense))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [ ]:
# Run the tuner
tuner = kt.RandomSearch(
    model_builder,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=1,
    directory='tuning_logs',
    project_name='kce_multi_feature_lstm'
)

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Start hyperparameter tuning
tuner.search(
    x_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop]
)

# ✅ Retrieve best model and hyperparameters
best_model = tuner.get_best_models(1)[0]
best_hp = tuner.get_best_hyperparameters(1)[0]

# ✅ Define output directory and file paths
output_dir = os.path.join("models", "tech")  
os.makedirs(output_dir, exist_ok=True)

model_file = os.path.join(output_dir, "KCE.BK_model.keras")
scaler_file = os.path.join(output_dir, "KCE.BK_scaler.pkl")

# ✅ Save best model and scaler
best_model.save(model_file)
with open(scaler_file, 'wb') as f:
    pickle.dump(scaler, f)

print(f"✅ Model and scaler saved to: {output_dir}")
print(f"Best LSTM Units: {best_hp.get('lstm_units')}")
print(f"Best Dropout Rate: {best_hp.get('dropout_rate')}")
print(f"Best Dense Units: {best_hp.get('dense_units')}")

In [370]:
test_predictions = best_model.predict(x_test)

# ⚠️ Rescale predictions and true labels
# If you used multiple features like ['Close', 'Volume'], pad the missing features with 0s before inverse_transform
num_features = scaler.n_features_in_

test_predictions_padded = np.concatenate(
    [test_predictions, np.zeros((len(test_predictions), num_features - 1))],
    axis=1
)
y_test_padded = np.concatenate(
    [y_test.reshape(-1, 1), np.zeros((len(y_test), num_features - 1))],
    axis=1
)

# Inverse transform back to original scale
test_predictions_rescaled = scaler.inverse_transform(test_predictions_padded)[:, 0]
y_test_rescaled = scaler.inverse_transform(y_test_padded)[:, 0]

mse = mean_squared_error(y_test_rescaled, test_predictions_rescaled)
mae = mean_absolute_error(y_test_rescaled, test_predictions_rescaled)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test_rescaled - test_predictions_rescaled) / y_test_rescaled)) * 100

print("\n📊 Evaluation Metrics:")
# print(f"   - Mean Squared Error (MSE): {mse:.4f}")
print(f"   - Mean Absolute Error (MAE): {mae:.4f}")
print(f"   - Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"   - Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 155ms/step

📊 Evaluation Metrics:
   - Mean Absolute Error (MAE): 0.3641
   - Root Mean Squared Error (RMSE): 0.5978
   - Mean Absolute Percentage Error (MAPE): 6.40%


In [91]:
# Generate predictions for 30-day forecast

# Take the last 60 days from the dataset for generating future predictions
last_60_days = scaled_data[-60:]

# Reshape last_60_days to fit the model input shape
x_future = last_60_days.reshape((1, last_60_days.shape[0], 1))

# Generate predictions
predictions = []
for _ in range(30):
    pred = model.predict(x_future) # Predict the next day
    predictions.append(pred[0,0]) # Append the prediction to the list
    
    # Update x_future by removing the first value and appending the prediction at the end
    x_future = np.append(x_future[:, 1:, :], np.array([[pred[0, 0]]]).reshape(1, 1, 1), axis=1)

# Inverse transform the predictions to get them back to the original scale
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

# Convert predictions to a DataFrame
future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=30)
predictions_df = pd.DataFrame(predictions, index=future_dates, columns=['Predicted Close'])
print(predictions_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━